In [2]:
import pandas as pd
import pymysql

conn = pymysql.connect(host='localhost', port=int(3306), user='root',passwd='1234', db='sakila')
cursor = conn.cursor(pymysql.cursors.DictCursor)

### Aggregate Functions

max()

min()

avg()

sum()

count()

> Aggregate Function을 사용할 땐 WHERE을 쓰면 오류가 난다.
> 
> WHERE우선 적용된뒤 GROUP BY가 이뤄지기 때문이다.
> 
> 따라서 `HAVING`을 사용해야한다.

In [12]:
sen = '''

SELECT customer_id, count(*)
FROM rental
GROUP BY customer_id
HAVING count(*) >= 40
ORDER BY 2 DESC;
'''

cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,customer_id,count(*)
0,148,46
1,526,45
2,144,42
3,236,42
4,75,41
5,197,40
6,469,40


In [7]:
sen = '''

SELECT MAX(amount) max_amt, MIN(amount) min_amt, AVG(amount) avg_amt, SUM(amount) tot_amt, COUNT(*) num_payments
FROM payment;

'''

cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,max_amt,min_amt,avg_amt,tot_amt,num_payments
0,11.99,0.00,4.200667,67416.51,16049


In [14]:
sen = '''

SELECT customer_id, MAX(amount) max_amt, MIN(amount) min_amt, AVG(amount) avg_amt, SUM(amount) tot_amt, COUNT(*) num_payments
FROM payment
GROUP BY customer_id;
'''

cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,customer_id,max_amt,min_amt,avg_amt,tot_amt,num_payments
0,1,9.99,0.99,3.708750,118.68,32
1,2,10.99,0.99,4.767778,128.73,27
2,3,10.99,0.99,5.220769,135.74,26
3,4,8.99,0.99,3.717273,81.78,22
4,5,9.99,0.99,3.805789,144.62,38
...,...,...,...,...,...,...
594,595,10.99,0.99,3.923333,117.70,30
595,596,6.99,0.99,3.454286,96.72,28
596,597,8.99,0.99,3.990000,99.75,25
597,598,7.99,0.99,3.808182,83.78,22


In [15]:
sen = '''

SELECT COUNT(customer_id) num_rows, COUNT(DISTINCT customer_id) num_customers
FROM payment;

'''

cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,num_rows,num_customers
0,16049,599


### datediff 
날짜를 반환함

In [17]:
sen = '''

SELECT MAX(datediff(return_date, rental_date)) coffee
FROM rental ; 
'''

cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,coffee
0,10


### Multicolumn Grouping

In [18]:
sen = '''

SELECT fa.actor_id, f.rating, count(*)
FROM film_actor fa
    INNER JOIN film f
    ON fa.film_id = f.film_id
GROUP BY fa.actor_id, f.rating
ORDER BY 1,2;

'''

cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,actor_id,rating,count(*)
0,1,G,4
1,1,PG,6
2,1,PG-13,1
3,1,R,3
4,1,NC-17,5
...,...,...,...
991,200,G,5
992,200,PG,3
993,200,PG-13,2
994,200,R,6


In [ ]:
sen = '''



'''

cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

In [20]:
sen = '''

SELECT extract(YEAR FROM rental_date) year,
    COUNT(*) how_many
FROM rental
GROUP BY year;


'''

cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,year,how_many
0,2005,15862
1,2006,182


### ROLL UP
Group by 할 때 total을 새로 만듬

actor_id 1에 대한 rating 전체의 합 = WITH ROLLUP

In [25]:
sen = '''

SELECT fa.actor_id, f.rating, count(*)
FROM film_actor fa
    INNER JOIN film f
    ON fa.film_id = f.film_id
GROUP BY fa.actor_id, f.rating WITH ROLLUP
ORDER BY 1,2;

'''

cursor.execute(sen)
pd.DataFrame(cursor.fetchall())

,actor_id,rating,count(*)
0,NaN,None,5462
1,1.0,None,19
2,1.0,G,4
3,1.0,NC-17,5
4,1.0,PG,6
...,...,...,...
1192,200.0,G,5
1193,200.0,NC-17,4
1194,200.0,PG,3
1195,200.0,PG-13,2


### Having 복습

When adding filters to a query that includes a group by clause,

think carefully about whether the filter acts on raw data, in which

case it belongs in the where clause, or on grouped data, in which

case it belongs in the having clause. 

In [30]:
sen = '''

SELECT fa.actor_id, f.rating, count(*)
FROM film_actor fa
    INNER JOIN film f
    ON fa.film_id = f.film_id
GROUP BY fa.actor_id, f.rating
HAVING count(*) > 9
ORDER BY 2;

'''

cursor.execute(sen)
# pd.DataFrame(cursor.fetchall())

48

In [36]:
# Exercise 8-1,2,3

sen = '''

SELECT customer_id, count(*), sum(amount)
FROM payment
GROUP BY customer_id
HAVING count(*) >= 40 ;
'''

cursor.execute(sen)
pd.DataFrame(cursor.fetchall())


,customer_id,count(*),sum(amount)
0,75,41,155.59
1,144,42,195.58
2,148,46,216.54
3,197,40,154.60
4,236,42,175.58
5,469,40,177.60
6,526,45,221.55
